# NOTES

Explanation of Changes:
Logarithmic Transformations: Applied to variables like GDP/capita and Tech Advancement CN to reduce skewness and deal with large values.

Lagged Variables: Created lagged versions of the GDP/capita and Tech Advancement CN to capture delayed effects.

Scaling: Reduced the magnitude of variables like Import, GDP/capita, and others to make coefficients more interpretable and to address scaling issues.

Model Comparison:

Conducted the Durbin-Wu-Hausman test to check whether the Random Effects model is more appropriate than the Fixed Effects model. If the p-value is low, you would reject Random Effects in favor of Fixed Effects.
Breusch-Pagan Test: Added a heteroscedasticity check to ensure the model does not suffer from heteroscedasticity.

In [ ]:
# -*- coding: utf-8 -*-
"""script-Copy1.ipynb
Automatically generated by Colab.

# IMPORTS AND SETTINGS
"""

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.api import add_constant
from linearmodels.panel import compare
import numpy as np
import scipy.stats
from linearmodels.panel import FirstDifferenceOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
import os
import warnings
warnings.filterwarnings('ignore')

project_folder = os.getcwd()

"""# DATA READING"""

# Read data
filename2 = os.path.join(project_folder, "Regression data", "panel_regression_absolute.csv")
data2 = pd.read_csv(os.path.join(project_folder, filename2))

# Set index and drop columns with all NaNs
data2 = data2.set_index(['Country', 'Year']).dropna(axis=1, how="all")
data2.head(20)

# Describe the dataset
data2.describe().T

"""# VARIABLE MANIPULATIONS"""

# Logarithmic transformations to handle large values and potential non-linearity
data2["log_GDP_per_capita"] = np.log(data2["GDP/capita"] + 1)
data2["log_Tech_Advancement_CN"] = np.log(data2["Tech Advancement CN"] + 1)

# Lagging variables by one year to account for delayed effects
data2["lag_GDP_per_capita"] = data2.groupby(level=0)["GDP/capita"].shift(1)
data2["lag_Tech_Advancement_CN"] = data2.groupby(level=0)["Tech Advancement CN"].shift(1)

# Drop missing rows after lagging
data2 = data2.dropna()

# Scaling to adjust for large magnitude differences
# Scaling down large variables for better interpretability
data2.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]] = data2.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]].div(1000)
data2["Import"] = data2["Import"].div(1000000)

# Show updated data
data2.describe().T

"""# MODELING"""

# Dependent and independent variables
data2["constant"] = 1
dependent_var = data2['Import'].to_frame()
independent_vars = data2[['log_GDP_per_capita', 'log_Tech_Advancement_CN', 'lag_GDP_per_capita', 'lag_Tech_Advancement_CN']]

# Check for multicollinearity with VIF
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
print(vif)

"""# FIXED EFFECTS MODEL"""

# Entity Fixed Effects Model (One-way)
fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
fe_results = fe_model.fit()
print("Fixed Effects Model Results:")
print(fe_results)

# Two-way Fixed Effects Model (Entity and Time effects)
two_way_fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True, drop_absorbed=True)
two_way_fe_results = two_way_fe_model.fit()
print("Two-Way Fixed Effects Model Results:")
print(two_way_fe_results)

"""# RANDOM EFFECTS MODEL"""

# Random Effects Model
re_model = RandomEffects(dependent_var, independent_vars)
re_results = re_model.fit()
print("\nRandom Effects Model Results:")
print(re_results)

"""# HAUSMAN TEST: FIXED VS RANDOM EFFECTS"""

# Perform Durbin-Wu-Hausman test to compare Fixed and Random Effects models
def durbin_wu_hausman_test(fe_results, re_results):
    u_fe = fe_results.resids
    u_re = re_results.resids
    dw_hausman_stat = np.sum((u_fe - u_re) ** 2) / np.sum(u_fe ** 2)
    df = fe_results.df_model - re_results.df_model
    p_value = 1 - scipy.stats.chi2.cdf(dw_hausman_stat, df)
    return dw_hausman_stat, p_value

dw_hausman_stat, p_value = durbin_wu_hausman_test(fe_results, re_results)
print("Durbin-Wu-Hausman Test Statistic:", dw_hausman_stat)
print("P-value:", p_value)

# If p-value is small (typically < 0.05), Fixed Effects is preferred over Random Effects

"""# BREUSCH-PAGAN TEST FOR HETEROSCEDASTICITY"""

# Breusch-Pagan test for heteroscedasticity
residuals = fe_results.resids
bp_test = het_breuschpagan(residuals, independent_vars)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))


# IMPORTS AND SETTINGS

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.api import add_constant
from linearmodels.panel import compare
import numpy as np
import scipy.stats
from linearmodels.panel import FirstDifferenceOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
# import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
from linearmodels.panel import PanelOLS
# from sklearn.preprocessing import StandardScaler
from statsmodels.stats.diagnostic import het_breuschpagan
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
project_folder=os.getcwd()
project_folder

'C:\\Users\\erott\\Documents\\Masterarbeit'

# SCRIPT

## DATA READING

In [ ]:
# data from Excel
filename2 = os.path.join(project_folder, "Regression data", "panel_regression_absolute.csv")
data2 = pd.read_csv(os.path.join(project_folder, filename2))
#'Country' as entity and 'Year' as time
data2 = data2.set_index(['Country', 'Year'])
data2=data2.dropna(axis=1, how="all")
data2.head(20)

Import    Fit  GDP/capita  Energy Consumption  \
Country Year                                                     
Austria 2005    1436678  0.672    38417.46              425.46   
        2006   41083867  0.678    40669.33              421.01   
        2007   20635660  0.521    46915.34              411.99   
        2008   28782325  0.559    51919.98              420.01   
        2009   18342478  0.530    48153.32              406.84   
        2010   66300152  0.434    46903.76              420.16   
        2011   54727543  0.438    51442.28              396.07   
        2012   96135630  0.338    48564.92              414.52   
        2013   66585288  0.230    50731.13              411.20   
        2014   38423535  0.000    51786.38              394.80   
        2015   33389376  0.000    44195.82              394.84   
        2016   22015038  0.000    45307.59              407.98   
        2017   20463038  0.000    47429.16              417.20   
        2018   26233065  0.000    51466.56              408.31   
        2019   63165749  0.000    50067.59              429.03   
Belgium 2005   38712939  0.000    36809.70              756.12   
        2006  175801095  0.000    38705.11              758.24   
        2007  182750887  0.000    44319.17              763.47   
        2008  405533516  0.000    48303.40              775.73   
        2009  606359750  0.000    44760.29              715.85   

              Annual Solar Capacity Addition  TechAdvancement   \
Country Year                                                     
Austria 2005                            -6.0               104   
        2006                             1.4               103   
        2007                             1.9               117   
        2008                             5.9               123   
        2009                            18.8                84   
        2010                            39.9                52   
        2011                            85.3                34   
        2012                           163.4                27   
        2013                           288.5                20   
        2014                           159.3                13   
        2015                           151.9                15   
        2016                           158.9                10   
        2017                           173.0                 7   
        2018                           186.2                11   
        2019                           247.0                 6   
Belgium 2005                             1.0                 3   
        2006                             0.0                 1   
        2007                            18.0                 3   
        2008                            42.0                 4   
        2009                           324.0                 4   

              Tech Advancement CN  Trade Policies EU  \
Country Year                                           
Austria 2005                  573               0.00   
        2006                  979               0.00   
        2007                 1621               0.00   
        2008                 2364               0.00   
        2009                 4149               0.00   
        2010                 6397               0.00   
        2011                 7634               0.00   
        2012                 8395               0.00   
        2013                 8057              21.84   
        2014                10136              47.70   
        2015                12974              47.70   
        2016                17340              47.70   
        2017                22199              47.70   
        2018                25048              33.79   
        2019                21697               0.00   
Belgium 2005                  573               0.00   
        2006                  979               0.00   
        2007                 1621              

In [ ]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 360 entries, ('Austria', 2005) to ('Sweden', 2019)
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Import                           360 non-null    int64  
 1   Fit                              360 non-null    float64
 2   GDP/capita                       360 non-null    float64
 3   Energy Consumption               360 non-null    float64
 4   Annual Solar Capacity Addition   360 non-null    float64
 5   TechAdvancement                  360 non-null    int64  
 6   Tech Advancement CN              360 non-null    int64  
 7   Trade Policies EU                360 non-null    float64
 8   Fixed Asset Investment CN (T-1)  360 non-null    float64
 9   Avg Wage Difference              360 non-null    float64
 10  Environ. St. Difference          360 non-null    float64
dtypes: float64(8), int64(3)
memory usage: 33.1+ KB


In [ ]:
# #Standardisation of data
# scaler = StandardScaler()
# numerical_data = data.select_dtypes(include=['float64', 'int64'])
# scaled_data = scaler.fit_transform(numerical_data)
# data_scaled = pd.DataFrame(scaled_data, columns=numerical_data.columns, index=data.index)
# data_scaled

In [ ]:
data2.describe().T

,count,mean,std,min,25%,50%,75%,max
Import,360.0,2.245835e+08,8.251732e+08,32139.0000,3.082600e+06,1.363522e+07,6.823544e+07,8.340234e+09
Fit,360.0,1.557222e-01,2.274972e-01,0.0000,0.000000e+00,0.000000e+00,2.232500e-01,8.300000e-01
GDP/capita,360.0,2.985638e+04,1.701686e+04,3899.8300,1.469957e+04,2.361370e+04,4.554015e+04,8.084830e+04
Energy Consumption,360.0,7.378902e+02,9.438475e+02,40.7500,1.879700e+02,3.415450e+02,8.358800e+02,4.062380e+03
Annual Solar Capacity Addition,360.0,3.226842e+02,1.035107e+03,-56.0000,2.000000e-01,1.125000e+01,1.414750e+02,9.539000e+03
TechAdvancement,360.0,5.721389e+01,1.537337e+02,0.0000,3.000000e+00,9.000000e+00,3.525000e+01,1.242000e+03
Tech Advancement CN,360.0,9.970867e+03,7.896539e+03,573.0000,2.364000e+03,8.057000e+03,1.734000e+04,2.504800e+04
Trade Policies EU,360.0,1.642867e+01,2.111868e+01,0.0000,0.000000e+00,0.000000e+00,4.770000e+01,4.770000e+01
Fixed Asset Investment CN (T-1),360.0,1.053947e+03,7.635221e+02,63.5100,2.114700e+02,1.219380e+03,1.800820e+03,2.284870e+03
Avg Wage Difference,360.0,1.813354e+03,1.252309e+03,-11.9100,7.755350e+02,1.416560e+03,2.904227e+03,6.615060e+03


In [ ]:
#scaling of the data
data2.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]] = data2.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]].div(1000)
data2.loc[:, "Import"] = data2.loc[:, "Import"].div(1000000)
data2.head(20)

Import    Fit  GDP/capita  Energy Consumption  \
Country Year                                                      
Austria 2005    1.436678  0.672    38.41746              425.46   
        2006   41.083867  0.678    40.66933              421.01   
        2007   20.635660  0.521    46.91534              411.99   
        2008   28.782325  0.559    51.91998              420.01   
        2009   18.342478  0.530    48.15332              406.84   
        2010   66.300152  0.434    46.90376              420.16   
        2011   54.727543  0.438    51.44228              396.07   
        2012   96.135630  0.338    48.56492              414.52   
        2013   66.585288  0.230    50.73113              411.20   
        2014   38.423535  0.000    51.78638              394.80   
        2015   33.389376  0.000    44.19582              394.84   
        2016   22.015038  0.000    45.30759              407.98   
        2017   20.463038  0.000    47.42916              417.20   
        2018   26.233065  0.000    51.46656              408.31   
        2019   63.165749  0.000    50.06759              429.03   
Belgium 2005   38.712939  0.000    36.80970              756.12   
        2006  175.801095  0.000    38.70511              758.24   
        2007  182.750887  0.000    44.31917              763.47   
        2008  405.533516  0.000    48.30340              775.73   
        2009  606.359750  0.000    44.76029              715.85   

              Annual Solar Capacity Addition  TechAdvancement   \
Country Year                                                     
Austria 2005                            -6.0               104   
        2006                             1.4               103   
        2007                             1.9               117   
        2008                             5.9               123   
        2009                            18.8                84   
        2010                            39.9                52   
        2011                            85.3                34   
        2012                           163.4                27   
        2013                           288.5                20   
        2014                           159.3                13   
        2015                           151.9                15   
        2016                           158.9                10   
        2017                           173.0                 7   
        2018                           186.2                11   
        2019                           247.0                 6   
Belgium 2005                             1.0                 3   
        2006                             0.0                 1   
        2007                            18.0                 3   
        2008                            42.0                 4   
        2009                           324.0                 4   

              Tech Advancement CN  Trade Policies EU  \
Country Year                                           
Austria 2005                0.573               0.00   
        2006                0.979               0.00   
        2007                1.621               0.00   
        2008                2.364               0.00   
        2009                4.149               0.00   
        2010                6.397               0.00   
        2011                7.634               0.00   
        2012                8.395               0.00   
        2013                8.057              21.84   
        2014               10.136              47.70   
        2015               12.974              47.70   
        2016               17.340              47.70   
        2017               22.199              47.70   
        2018               25.048              33.79   
        2019               21.697               0.00   
Belgium 2005                0.573               0.00   
        2006                0.979               0.00   
        2007              

In [ ]:
data2.describe().T

,count,mean,std,min,25%,50%,75%,max
Import,360.0,224.583454,825.173238,0.032139,3.082600,13.635224,68.235437,8340.234422
Fit,360.0,0.155722,0.227497,0.000000,0.000000,0.000000,0.223250,0.830000
GDP/capita,360.0,29.856376,17.016865,3.899830,14.699565,23.613700,45.540148,80.848300
Energy Consumption,360.0,737.890194,943.847525,40.750000,187.970000,341.545000,835.880000,4062.380000
Annual Solar Capacity Addition,360.0,322.684167,1035.107133,-56.000000,0.200000,11.250000,141.475000,9539.000000
TechAdvancement,360.0,57.213889,153.733667,0.000000,3.000000,9.000000,35.250000,1242.000000
Tech Advancement CN,360.0,9.970867,7.896539,0.573000,2.364000,8.057000,17.340000,25.048000
Trade Policies EU,360.0,16.428667,21.118675,0.000000,0.000000,0.000000,47.700000,47.700000
Fixed Asset Investment CN (T-1),360.0,1.053947,0.763522,0.063510,0.211470,1.219380,1.800820,2.284870
Avg Wage Difference,360.0,1.813354,1.252309,-0.011910,0.775535,1.416560,2.904228,6.615060


In [ ]:
# dependent, independent and control variables
data2["constant"]=1
dependent_var = data2['Import'].to_frame()
independent_vars = data2.iloc[:,1:]
dependent_var.shape, independent_vars.shape

((360, 1), (360, 11))

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
vif

Variance Inflation Factors (VIF):


,VIF Factor,features
0,1.315725,Fit
1,4.544590,GDP/capita
2,2.853626,Energy Consumption
3,2.384757,Annual Solar Capacity Addition
4,3.693410,TechAdvancement
5,8.957944,Tech Advancement CN
6,2.102597,Trade Policies EU
7,15.198968,Fixed Asset Investment CN (T-1)
8,5.154516,Avg Wage Difference
9,4.278517,Environ. St. Difference


In [ ]:
correlation_matrix = data2.corr()
print(correlation_matrix)

                                   Import       Fit  GDP/capita  \
Import                           1.000000  0.266253    0.166235   
Fit                              0.266253  1.000000    0.013715   
GDP/capita                       0.166235  0.013715    1.000000   
Energy Consumption               0.522634  0.292713    0.284049   
Annual Solar Capacity Addition   0.856715  0.177242    0.177018   
TechAdvancement                  0.794065  0.264630    0.194313   
Tech Advancement CN             -0.079013 -0.364941    0.064757   
Trade Policies EU               -0.147700 -0.300296    0.011244   
Fixed Asset Investment CN (T-1) -0.090597 -0.375513    0.060157   
Avg Wage Difference              0.230875 -0.011542    0.875215   
Environ. St. Difference          0.127749  0.349210    0.105704   
constant                              NaN       NaN         NaN   

                                 Energy Consumption  \
Import                                     0.522634   
Fit               

In [ ]:
# Verify column names
print(independent_vars.columns)

# Drop the column if it exists
independent_vars = independent_vars.drop(columns=["constant","Fixed Asset Investment CN (T-1)","Avg Wage Difference"], errors='ignore')

# Verify the result
print(independent_vars.columns)

Index(['Fit', 'GDP/capita', 'Energy Consumption',
       'Annual Solar Capacity Addition', 'TechAdvancement ',
       'Tech Advancement CN', 'Trade Policies EU',
       'Fixed Asset Investment CN (T-1)', 'Avg Wage Difference',
       'Environ. St. Difference', 'constant'],
      dtype='object')
Index(['Fit', 'GDP/capita', 'Energy Consumption',
       'Annual Solar Capacity Addition', 'TechAdvancement ',
       'Tech Advancement CN', 'Trade Policies EU', 'Environ. St. Difference'],
      dtype='object')


In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
vif

Variance Inflation Factors (VIF):


,VIF Factor,features
0,1.805317,Fit
1,4.394710,GDP/capita
2,4.227952,Energy Consumption
3,2.518774,Annual Solar Capacity Addition
4,4.078471,TechAdvancement
5,3.899547,Tech Advancement CN
6,2.914940,Trade Policies EU
7,3.029983,Environ. St. Difference


## MODELS

### FE MODEL (ENTITY ONE WAY)

In [ ]:
print(independent_vars.columns.duplicated())

[False False False False False False False False]


In [ ]:
corr_matrix = independent_vars.corr()
print(corr_matrix)

                                     Fit  GDP/capita  Energy Consumption  \
Fit                             1.000000    0.013715            0.292713   
GDP/capita                      0.013715    1.000000            0.284049   
Energy Consumption              0.292713    0.284049            1.000000   
Annual Solar Capacity Addition  0.177242    0.177018            0.568172   
TechAdvancement                 0.264630    0.194313            0.758884   
Tech Advancement CN            -0.364941    0.064757           -0.021697   
Trade Policies EU              -0.300296    0.011244           -0.020441   
Environ. St. Difference         0.349210    0.105704            0.137263   

                                Annual Solar Capacity Addition  \
Fit                                                   0.177242   
GDP/capita                                            0.177018   
Energy Consumption                                    0.568172   
Annual Solar Capacity Addition                     

In [ ]:
def matrix_rank_check(df):
    return np.linalg.matrix_rank(df.values)

print(f"Rank of independent variables matrix: {matrix_rank_check(independent_vars)}")
print(f"Number of columns in independent variables: {independent_vars.shape[1]}")

Rank of independent variables matrix: 8
Number of columns in independent variables: 8


In [ ]:
print(corr_matrix)

                                     Fit  GDP/capita  Energy Consumption  \
Fit                             1.000000    0.013715            0.292713   
GDP/capita                      0.013715    1.000000            0.284049   
Energy Consumption              0.292713    0.284049            1.000000   
Annual Solar Capacity Addition  0.177242    0.177018            0.568172   
TechAdvancement                 0.264630    0.194313            0.758884   
Tech Advancement CN            -0.364941    0.064757           -0.021697   
Trade Policies EU              -0.300296    0.011244           -0.020441   
Environ. St. Difference         0.349210    0.105704            0.137263   

                                Annual Solar Capacity Addition  \
Fit                                                   0.177242   
GDP/capita                                            0.177018   
Energy Consumption                                    0.568172   
Annual Solar Capacity Addition                     

In [ ]:
# Fixed Effects Model
fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
fe_results = fe_model.fit()

print("Fixed Effects Model Results:")
fe_results

Fixed Effects Model Results:


Dep. Variable:,Import,R-squared:,0.7987
Estimator:,PanelOLS,R-squared (Between):,-32.646
No. Observations:,360,R-squared (Within):,0.7987
Date:,"Fri, Jul 12 2024",R-squared (Overall):,-13.433
Time:,18:09:46,Log-likelihood,-2552.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,162.65
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(8,328)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,162.65


### FE MODEL (TWO-WAY)

In [ ]:
#two-way fixed effects model
two_model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True,  drop_absorbed = True)
results = two_model.fit()
results

Dep. Variable:,Import,R-squared:,0.7818
Estimator:,PanelOLS,R-squared (Between):,-36.624
No. Observations:,360,R-squared (Within):,0.7813
Date:,"Fri, Jul 12 2024",R-squared (Overall):,-15.136
Time:,18:09:46,Log-likelihood,-2543.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,188.71
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(6,316)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,188.71


### RANDOM EFFECTS

In [ ]:
# Random Effects Model
re_model = RandomEffects(dependent_var, independent_vars)
re_results = re_model.fit()

print("\nRandom Effects Model Results:")
re_results


Random Effects Model Results:


Dep. Variable:,Import,R-squared:,0.8375
Estimator:,RandomEffects,R-squared (Between):,0.9583
No. Observations:,360,R-squared (Within):,0.7484
Date:,"Fri, Jul 12 2024",R-squared (Overall):,0.8377
Time:,18:09:46,Log-likelihood,-2613.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,226.76
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(8,352)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,226.76


In [ ]:
#Hausmann Test
comparison = compare({"Fixed": fe_results, "Random": re_results})
comparison

,Fixed,Random
Dep. Variable,Import,Import
Estimator,PanelOLS,RandomEffects
No. Observations,360,360
Cov. Est.,Unadjusted,Unadjusted
R-squared,0.7987,0.8375
R-Squared (Within),0.7987,0.7484
R-Squared (Between),-32.646,0.9583
R-Squared (Overall),-13.433,0.8377
F-statistic,162.65,226.76
P-value (F-stat),0.0000,0.0000


### Durbin-Wu-Hausman test

In [ ]:
import numpy as np
from statsmodels.regression.linear_model import OLS
from scipy.stats import chi2

def durbin_wu_hausman_test(ols_fixed, ols_random):
    """
    Perform the Durbin-Wu-Hausman test for endogeneity.

    Parameters:
    ols_fixed : OLS
        Fixed-effects regression results
    ols_random : OLS
        Random-effects regression results

    Returns:
    chi_squared_stat : float
        The chi-squared test statistic
    p_value : float
        The p-value of the test
    """
    # Residuals from fixed effects model
    u_fe = ols_fixed.resids

    # Residuals from random effects model
    u_re = ols_random.resids

    # Calculate the test statistic
    dw_hausman_stat = np.sum((u_fe - u_re)**2) / np.sum(u_fe**2)

    df = fe_results.df_model - re_results.df_model

    # Compute the p-value
    p_value = 1 - chi2.cdf(dw_hausman_stat, df)

    return dw_hausman_stat, p_value

# Example usage:
# Assuming you have panel data stored in a DataFrame called 'data'
# with columns 'dependent_var', 'independent_var1', 'independent_var2', etc.
# and a variable 'id' for individual identifiers and 'time' for time identifiers.


# Perform the Durbin-Wu-Hausman test
dw_hausman_stat, p_value = durbin_wu_hausman_test(fe_results, re_results)

print("Durbin-Wu-Hausman Test Statistic:", dw_hausman_stat)
print("P-value:", p_value)

Durbin-Wu-Hausman Test Statistic: 0.4026918013803389
P-value: 1.0


In [ ]:
fe_results

Dep. Variable:,Import,R-squared:,0.7987
Estimator:,PanelOLS,R-squared (Between):,-32.646
No. Observations:,360,R-squared (Within):,0.7987
Date:,"Fri, Jul 12 2024",R-squared (Overall):,-13.433
Time:,18:09:46,Log-likelihood,-2552.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,162.65
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(8,328)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,162.65


In [ ]:
#Wald Test: H0 is that all the regressors are not associated (no effect) with the dependent variables, H0 model has only the intercept (unconditional average)
wald_result = fe_results.wald_test(formula="Fit=0, `GDP/capita`=0, `Energy Consumption`=0, `Annual Solar Capacity Addition`=0, `Tech Advancement CN`=0, `TechAdvancement `=0, `Trade Policies EU`=0, `Environ. St. Difference`=0")
wald_result

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 1301.1621
P-value: 0.0000
Distributed: chi2(8)
WaldTestStatistic, id: 0x1db5869e2a0

In [ ]:
# Perform the Breusch-Pagan test
residuals = fe_results.resids
independent_vars_bp_test = independent_vars.copy()
independent_vars_bp_test["constant"]=1
independent_vars_bp_test

bp_test = het_breuschpagan(resid=residuals, exog_het=independent_vars_bp_test, robust=True)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))

{'Lagrange multiplier statistic': 71.49189800125988, 'p-value': 2.4785064769839843e-12, 'f-value': 10.872162698637055, 'f p-value': 1.0472299529299159e-13}
